# MNIST Regression
We often solve the MNIST problem by using different kinds of classification techniques. Of cause MNIST classifcation can achieve a better result, how about solving MNIST problem using Regression? I will try it in this notebook.
## Importing Packages

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Loading the data

In [ ]:
train_pd = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_pd = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
train = np.array(train_pd)
test = np.array(test_pd)
train_images = train[:, 1:].reshape((-1, 28, 28, 1)) / 255.0
train_labels = train[:, 0].astype(np.uint8)
test_images = test.reshape((-1, 28, 28, 1)) / 255.0

Define a special RELU Layer with max value.

In [ ]:
class RELUWithMaxValue(tf.keras.layers.Layer):
    def __init__(self, max_value):
        super(RELUWithMaxValue, self).__init__()
        self.max_value = max_value
    def call(self, inputs):
        return tf.keras.backend.relu(inputs, max_value=self.max_value)

## Model Development

Define a special version of accuracy metric that can calculate accuracy corretly.

In [ ]:
def accuracy(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.round(y_pred) == y_true, dtype=tf.float32))

### CNN Model

In [ ]:
def get_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(2, 2),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        BatchNormalization(),
        Flatten(),
        Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
        Dense(1),
        RELUWithMaxValue(9.0)
    ])
    model.compile(
        optimizer='adam',
        loss='mean_squared_error',
        metrics=[accuracy]
    )
    return model

In [ ]:
cnn_model = get_cnn_model()
checkpoint_filepath = "best_checkpoint"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor="val_accuracy",
    mode="max"
)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels)
history = cnn_model.fit(train_images, train_labels, epochs=100, validation_data=(val_images, val_labels), callbacks=[model_checkpoint, early_stopping], verbose=1)
pd.DataFrame(history.history).plot()
plt.show()
cnn_model = get_cnn_model()
cnn_model.load_weights(checkpoint_filepath)

## Submission
Although I solve this problem using Regression techniques, calculating test scores still requires converting results to integers.

In [ ]:
test_labels = np.array(np.round(cnn_model.predict(test_images)), dtype=int).reshape(-1)
print(test_labels.shape)

In [ ]:
pd.DataFrame(test_labels).hist()

Here is first 100 item of test_labels.

In [ ]:
print(test_labels[:100])

In [ ]:
image_ids = np.arange(1, test_labels.shape[0]+1)
result = np.concatenate((image_ids.reshape(-1, 1), test_labels.reshape(-1, 1)), axis=1)
df = pd.DataFrame(result, columns=["ImageId", "Label"], dtype='int')
df.to_csv("submission.csv", index=False)

## Conclusion

Currently MNIST Regression can achieve 92% test accuracy which is not bad, there must be more ways to improve it. Any suggesion? I am glad to hear from you.